## Part 1: Preprocessing

In [35]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [36]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [37]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [38]:
# Create a list of at least 10 column names to use as X data
X_cols = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
          'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
          'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_cols]

# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [39]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [40]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [41]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [42]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = dept_encoder.transform(y_train[['Department']])
y_test_dept = dept_encoder.transform(y_test[['Department']])
print(y_train_dept)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [43]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])
print(y_train_attrition)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Part 2: Create, Compile, and Train the Model

In [44]:
# Find the number of columns in the X training data.
number_input_features = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,))

# Create at least two shared layers
shared_layer1 = layers.Dense(128, activation="relu")(input_layer)
shared_layer2 = layers.Dense(64, activation="relu")(shared_layer1)

In [45]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_layer = layers.Dense(32, activation="relu")(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation="softmax", name="department_output")(department_layer)

In [46]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_layer = layers.Dense(32, activation="relu")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation="softmax", name="attrition_output")(attrition_layer)

In [47]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])
# Compile the model
model.compile(optimizer="adam",
              loss={"department_output": "categorical_crossentropy",
                    "attrition_output": "categorical_crossentropy"},
             metrics={"department_output": "accuracy", "attrition_output": "accuracy"})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 11)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 128)            │          1,536 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 64)             │          8,256 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 32)             │          2,080 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          2,080 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,117 (55.14 KB)

 Trainable params: 14,117 (55.14 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Train the model
model.fit(x=X_train_scaled,
          y={"department_output": y_train_dept, "attrition_output": y_train_attrition},
          epochs=100,
          batch_size=128,
          shuffle=True,
          verbose=2)

Epoch 1/100
10/10 - 5s - 493ms/step - attrition_output_accuracy: 0.7560 - attrition_output_loss: 0.5588 - department_output_accuracy: 0.5595 - department_output_loss: 0.9902 - loss: 1.5748
Epoch 2/100
10/10 - 0s - 19ms/step - attrition_output_accuracy: 0.8316 - attrition_output_loss: 0.4571 - department_output_accuracy: 0.6505 - department_output_loss: 0.8468 - loss: 1.2852
Epoch 3/100
10/10 - 0s - 16ms/step - attrition_output_accuracy: 0.8308 - attrition_output_loss: 0.4033 - department_output_accuracy: 0.6505 - department_output_loss: 0.7896 - loss: 1.2137
Epoch 4/100
10/10 - 0s - 28ms/step - attrition_output_accuracy: 0.8444 - attrition_output_loss: 0.3901 - department_output_accuracy: 0.6505 - department_output_loss: 0.7883 - loss: 1.1680
Epoch 5/100
10/10 - 0s - 35ms/step - attrition_output_accuracy: 0.8478 - attrition_output_loss: 0.3944 - department_output_accuracy: 0.6522 - department_output_loss: 0.7666 - loss: 1.1485
Epoch 6/100
10/10 - 0s - 27ms/step - attrition_output_accur

In [49]:
# Evaluate the model with the testing data
model_loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(
    x=X_test_scaled, y={"department_output": y_test_dept, "attrition_output": y_test_attrition}, verbose=2
)

10/10 - 0s - 43ms/step - attrition_output_accuracy: 0.8197 - attrition_output_loss: 0.7448 - department_output_accuracy: 0.5714 - department_output_loss: 1.1029 - loss: 1.9054


In [50]:
# Print the accuracy for both department and attrition
print(f"Department Loss: {department_loss}, Department Accuracy: {department_accuracy}")
print(f"Attrition Loss: {attrition_loss}, Attrition Accuracy: {attrition_accuracy}")

Department Loss: 1.1028629541397095, Department Accuracy: 0.819727897644043
Attrition Loss: 0.744828999042511, Attrition Accuracy: 0.5714285969734192


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

## YOUR ANSWERS HERE

1. ### Is Accuracy the Best Metric?

  Relying on accuracy as the sole metric may not be ideal, especially when dealing with imbalanced classes. In the context of attrition prediction, which often sees such imbalances, metrics like precision, recall, and the F1 score can provide deeper insights.



2. ### Chosen Activation Functions:

- **Softmax:** Utilized for department prediction due to its suitability for multi-class classification.
- **Sigmoid:** Best for attrition prediction since it is a binary classification (Yes/No).


3. ### Ways to Improve the Model:

- Incorporate cross-validation to enhance generalization.
- Implement techniques to address class imbalance, such as SMOTE or class weighting.
- Perform hyperparameter tuning, focusing on layers, nodes, epochs, and learning rates.
- Experiment with advanced neural architectures or ensemble methods for better performance.